In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
from Connections import Connections



In [2]:
bd = Connections()

In [3]:
df = bd.get_orders_data()

In [ ]:
df["order_created_at_hour"] = pd.to_datetime(df["order_created_at_hour"])
df.set_index('order_created_at_hour', inplace=True)

Analisando os dados Temporários

In [ ]:
df.plot()

### Verificando Stationary

Stationarity is an important concept in time-series and any time-series data should undergo a stationarity test before proceeding with a model.

We use the ‘Augmented Dickey-Fuller Test’ to check whether the data is stationary or not which is available in the ‘pmdarima’ package.

In [ ]:
adf_test = ADFTest(alpha=0.5)
adf_test.should_diff(df)

From the above, we can conclude that the data is non-stationary. Hence, we would need to use the “Integrated (I)” concept, denoted by value ‘d’ in time series to make the data stationary while building the Auto ARIMA model

###  Train and Test split

Split into train and test datasets to build the model on the training dataset and forecast using the test dataset.

In [ ]:
train, test= np.split(df, [int(.70 * len(df))])

plt.plot(train)
plt.plot(test)

In [ ]:

modelo_auto_arima = auto_arima(train, m = 12, seasonal = True, trace = False , error_action = 'ignore',
                                suppres_warnings=True, stepwise = False)


In [ ]:
modelo_auto_arima.summary()

In [ ]:
df_mod = modelo_auto_arima.predict(n_periods=429)
prediction = pd.DataFrame(df_mod)
prediction.columns = ["predict_orders"]
prediction.index = test.index


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train, label="Training")
plt.plot(test, label="Test")
plt.plot(prediction, label="Prediction")

plt.show()

### Verificando R2 

In [ ]:
from sklearn.metrics import r2_score
result = pd.concat([test, prediction["predict_orders"]], axis=1)

teste = result
r2_score(teste["order_count"], teste["predict_orders"])

### Biblioteca Ploty 

In [ ]:
train.rename(columns={'order_count': 'order_count_train'}, inplace=True)

concat_df = pd.concat([train,test, prediction["predict_orders"]], axis=1)


In [ ]:
fig = px.line(concat_df, x=concat_df.index, y=["order_count_train","predict_orders","order_count"], template = "plotly_dark")
fig.show()